In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import shutil
from concurrent.futures import ThreadPoolExecutor, as_completed
from ase.db import connect
from ase.io import write
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core.surface import generate_all_slabs

def is_low_index(miller_index, limit=3):
    """Check if each component of the Miller index does not exceed the limit (absolute value)"""
    return all(abs(x) <= limit for x in miller_index)

def process_slab(slab_pmg, idx, row_id, enlarge_factor, out_prefix, folder):
    try:
        try:
            slab_pmg = slab_pmg.get_orthogonal_c_slab()
        except Exception as e:
            print(f"[WARN] Orthogonalization failed for surface {slab_pmg.miller_index}, error: {e}")
            # Continue with original structure if orthogonalization fails

        if enlarge_factor > 1:
            supercell_matrix = [[enlarge_factor, 0, 0],
                                [0, enlarge_factor, 0],
                                [0, 0, 1]]
            slab_pmg = slab_pmg.copy()
            slab_pmg.make_supercell(supercell_matrix)

        slab_ase = AseAtomsAdaptor.get_atoms(slab_pmg)
        z_min = slab_ase.positions[:, 2].min()
        slab_ase.positions[:, 2] -= z_min

        miller_str = "".join(str(int(x)) for x in slab_pmg.miller_index)
        file_name = f"{out_prefix}_ID{row_id}_{miller_str}_{idx}.vasp"
        full_path = os.path.join(folder, file_name)
        write(full_path, slab_ase, format='vasp')

        try:
            surf_sites = slab_pmg.get_surface_sites(tag=True)
            top_count = len(surf_sites.get("top", []))
            bottom_count = len(surf_sites.get("bottom", []))
            max_count = max(top_count, bottom_count)
            area = slab_pmg.surface_area
            density = max_count / area if area > 0 else 0
        except Exception as e:
            print(f"[WARN] Surface density analysis failed for surface {slab_pmg.miller_index}: {e}")
            density = 0

        return slab_pmg.miller_index, density, full_path, idx
    except Exception as exc:
        print(f"[ERROR] Fatal error encountered while processing slab (idx={idx}): {exc}")
        return None, 0, None, idx

def generate_unique_slabs_from_db(
    db_file="materials.db",
    selection=None,
    max_index=2,
    min_slab_size=8.0,
    min_vacuum_size=10.0,
    out_prefix="slab",
    enlarge_factor=1,
    max_workers=4
):
    db = connect(db_file)
    rows = db.select(selection) if selection else db.select()

    for row in rows:
        bulk_atoms = row.toatoms()
        formula = bulk_atoms.get_chemical_formula()
        model_name = getattr(row, "name", formula)
        print(f"\n[INFO] Processing database entry ID={row.id}, model name={model_name}, Formula={formula}")
        
        folder = os.path.join(os.getcwd(), model_name)
        os.makedirs(folder, exist_ok=True)

        pmg_bulk = AseAtomsAdaptor.get_structure(bulk_atoms)

        slabs = generate_all_slabs(
            structure=pmg_bulk,
            max_index=max_index,
            min_slab_size=min_slab_size,
            min_vacuum_size=min_vacuum_size,
            in_unit_planes=False,
            primitive=True,
            max_normal_search=None,
            symmetrize=True,
        )

        if not slabs:
            print("    -> No slabs generated (structure may be too complex or highly symmetric).")
            continue

        print(f"    -> Generated {len(slabs)} symmetrically inequivalent surfaces (hkl).")

        tasks = []
        results = []  # Store (miller_index, density, file_path, idx)
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            for i, slab_pmg in enumerate(slabs, start=1):
                future = executor.submit(process_slab, slab_pmg, i, row.id,
                                           enlarge_factor, out_prefix, folder)
                tasks.append(future)

            for future in as_completed(tasks):
                miller, density, file_path, idx = future.result()
                if file_path is not None:
                    results.append((miller, density, file_path, idx))
                    print(f"       -> Processing complete, surface {miller}, surface density = {density:.4f}, file: {file_path}")
                else:
                    print("[WARN] A slab processing failed, skipping.")

        # For each Miller index, copy the one with highest surface density as best file
        best_per_miller = {}
        for miller, density, file_path, idx in results:
            if miller in best_per_miller:
                if density > best_per_miller[miller][0]:
                    best_per_miller[miller] = (density, file_path, idx)
            else:
                best_per_miller[miller] = (density, file_path, idx)

        for miller, (density, file_path, idx) in best_per_miller.items():
            miller_str = "".join(str(int(x)) for x in miller)
            dest_file = os.path.join(folder, f"best_ID{row.id}_{miller_str}_{idx}.vasp")
            shutil.copy(file_path, dest_file)
            print(f"    -> Best surface for Miller {miller}: {file_path}, copied as: {dest_file}")

        # Select the overall best surface with highest density among all surfaces and copy as file starting with totalbest_
        if results:
            overall_best = max(results, key=lambda x: x[1])
            best_miller_str = "".join(str(int(x)) for x in overall_best[0])
            best_idx = overall_best[3]
            best_dest = os.path.join(folder, f"totalbest_ID{row.id}_{best_miller_str}_{best_idx}.vasp")
            shutil.copy(overall_best[2], best_dest)
            print(f"    -> Overall best surface for model {model_name}: {overall_best[2]}")
            print(f"       Copied as: {best_dest}")


def main():
    generate_unique_slabs_from_db(
        db_file='initial_db.db',
        selection=None,
        max_index=2,
        min_slab_size=8.0,
        min_vacuum_size=10.0,
        out_prefix='slab',
        enlarge_factor=1,
        max_workers=16
    )

if __name__ == "__main__":
    main()